In [2]:
import json
import re
from docx import Document

In [ ]:
doc = Document('regions.docx')
table = doc.tables[0]
regions = {}
for row in table.rows:
    regions[row.cells[0].text] = row.cells[1].text


In [ ]:
with open('logins.txt', 'r') as file_from:
    curr_addresses = []
    line = file_from.readline()
    while line:
        curr_addresses.append(line)
        line = file_from.readline()

In [ ]:
with open('addresses_details.json', 'w') as json_addresses:
    data = {}
    for school in curr_addresses:
        data[school[0]] = {}
        data[school[0]]['source_address'] = school[1]
        data[school[0]]['final_address'] = []
        data[school[0]]['locality'] = school[2]
        data[school[0]]['region'] = regions[school[0][3:5]]
    
    json.dump(data, json_addresses, indent=4, ensure_ascii=False)

In [ ]:
patt_list = ['(\d{6})?[,\. ]*',
        '(Россия|Российская Федерация|РФ)?[,\. ]*',
        '(\d{6})?[,\. ]*',
        '((?:(?:Республика|Респ\.|Респ) [а-яА-Я-]*)|[а-яА-Я-]* (?:Республика|Респ\.|Респ)|(?:РСО-Алания)|Р(?:[А-Я]|МЭ)|УР|ЧР|КБР)?[,\. ]*',
        '((?:Ямало-Ненецкий автономный округ)|ЯНАО)?[, ]*'
        '((?:[а-яА-Я-]* (?:Область|область|обл\.|обл))|(?:(?:ОБЛАСТЬ|Область|область|обл\.|обл) [а-яА-Я-]*))?[, ]*',
        '([а-яА-Я-]* край)?[, ]*',
        '(\d{6})?[, ]*',
        '([а-яА-Я-]* национальный район)?[, ]*', 
        '((?:(?:муниципальный)? ?района|район|р-н|р-он) (?:имени|[а-яА-Я-]* ?){1,3}|[а-яА-Я-]* (?:(?:муниципальный)? ?район|р-н|р-он)|[а-яА-Я-]* кожуун)?[, ]*',
        '([А-Я][А-Яа-я-]+ улус)?[, ]*',
        '((?:[а-яА-Я-]* городской округ)|(?:(?:(?:Городской округ)|(?:городской округ)|г.о.|Г.о.|Г.О.|г.О.) ?[ёа-яА-Я-]*))?[, ]*',
        '((?:ГОРОД |Г\. ?|Г |г\. ?|город |г )(?:[А-Я][\.ёа-яА-Я-]* ?)*)?[, ]*', 
        '([А-Яа-я-]* территориальный округ)?[, ]*',
        '((?:[А-Я][\.ёа-яА-Я-]* ?){1,3}(?:г\.|город|г))?[, ]*',
        '([а-яА-Я-]* (?:(?:муниципальный)? ?район|р-н))?[, ]*',
        '((?:(?:РАБОЧИЙ ПОСЕЛОК) |(?:рабочий поселок) |р\.п\. ?|сельсовет |станица |ст-ца |ст |ст\. ?|аал |аул |а\. ?|а |с\.п\. ?|с\. ?|\.с |село |с |д\. ?|деревня |дер\. ?|дер |д |п\.г\.т\. ?|п |пос\. ?|пос |п\. ?|пгт |пгт\. ?|пос(?:е|ё)лок |поселение |селение |х |х\. ?|хутор |у |у\. ?|улус )(?:[А-Я][\.ёа-яА-Я-]* ?)*)?[, ]*', 
        '((?:[А-Я][\.а-яА-Я-]* ?){1,3} ?(?:пгт|станица|сельсовет|ст-ца|ст|ст\.|аал|аул|а\.|а|с\.|с|.с|село|д\.|деревня|д|пос(?:е|ё)лок|поселение|селение|пос\.|пос|п\.|пгт\.|п|х\.|хутор|х|у\.|улус))?[, ]*',
        '((?:[ёа-яА-Я-]* микрорайон|м-н|мкр.|мкрн.|микр\.|микр)|(?:микрорайон|м-н|мкр.|мкрн.|микр\.|микр) ?(?:(?:[0-9]{1,3}|[А-Я][\.а-яА-Я-]+|(?:[0-9]+(?:-ой|-ый|-ий|-й|-Й))|лет|им\.|им|и) ?)*)?[, ]*',
        '((?:Улица |УЛИЦА |УЛ\. ?|УЛ |квартал |кв\. ?|кв |ул\. ?|ул |улица |улицв |улици |проспект |пр-т |площадь |пл\. ?|пл |пер\. ?|пер |пр-д |проезд |пр |пр\. ?|переулок |б-р |бульвар |шоссе |ш\. ?|ш )(?:(?:[А-Я][\.ёа-яА-Я-]+|(?:[0-9]+(?:-ая|-ья|-я))|[0-9]+|[0-9]{1,3}-летия|летия|лет|им\.|им|и|кольцо|мая|спуск|шоссе) ?)*)?[, ]*',
        '((?:(?:летия|лет|им\.|им|и|кольцо|мая|[\.ёа-яА-Я-]+|[0-9]+(?:-ая|-ья|-я)|[0-9]+) ?){1,4}(?:кв\.|кв|ул\.|улица|улици|ул|у|проспект|пр-т|пр-кт|площадь|пер\.|пер|пр-д|проезд|пр|пр\.|переулок|б-р|бульвар|шоссе|ш\.|ш))?[, ]*',
        '((?:ДОМ|дом|д\.|д|владение)? ?№? ?(?:\d{0,3})?-?)?[, ]*',
        '((?:корп\.|корпус|корп|строение|здание) ?.*)?[, ]*$']


In [ ]:
json_addresses = open('addresses_details.json', 'r+')
data = json.load(json_addresses)
json_addresses.close()

for patt in patt_list:
    amount = 0
    
    for key in data:
        patt = re.compile(patt)
        res = re.match(patt, data[key]['source_address'])
        if res is not None:
            matching = data[key]['source_address'][res.start():res.end()]
            if matching != '':
                data[key]['final_address'].append(data[key]['source_address'][res.start():res.end()])
            new_str = data[key]['source_address'][res.end():]
            data[key]['source_address'] = new_str
            amount += 1

with open('addresses_details.json', 'r+') as json_addresses:
    json.dump(data, json_addresses, ensure_ascii=False, indent=4)


In [ ]:
json_addresses = open('addresses_details.json', 'r+')
data = json.load(json_addresses)

with open('resulted_strings.txt', 'w') as file_to:
    for login in data:
        file_to.write(login + ' ' + ', '.join(data[login]['final_address']))